# Set Up

In [ ]:
# Input each setting value of Azure Data Storage 
MOUNTPOINT = "/mnt/termpaper"

if MOUNTPOINT in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
  dbutils.fs.unmount(MOUNTPOINT)

STORAGE_ACCOUNT = "cs777"
CONTAINER = "termpaper"
SASTOKEN = dbutils.secrets.get(scope="tomo0530", key="storageaccess")

SOURCE = "wasbs://{container}@{storage_acct}.blob.core.windows.net/".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
URI = "fs.azure.sas.{container}.{storage_acct}.blob.core.windows.net".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)

/mnt/termpaper has been unmounted.

In [ ]:
# Mount Azure Data Strorage on Databricks
try:
  dbutils.fs.mount(
    source=SOURCE,
    mount_point=MOUNTPOINT,
    extra_configs={URI:SASTOKEN})
except Exception as e:
  if "Directory already mounted" in str(e):
    pass # Ignore error if already mounted.
  else:
    raise e
print("Success.")

Success.

In [ ]:
# Set the file path
file1 = MOUNTPOINT + "/a9a.txt"
file2 = MOUNTPOINT + "/kddb-raw-libsvm.t.bz2"
file3 = MOUNTPOINT + "/kdda.bz2"

In [ ]:
from functools import wraps
import math
import time
from typing import Tuple, Dict

import numpy as np
import pandas as pd

from sklearn.datasets import load_svmlight_file
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

import pyspark.sql
from pyspark.sql import SparkSession, Column
from pyspark.ml.feature import ChiSqSelector
from pyspark.sql.functions import *
from pyspark.ml.linalg import *

In [ ]:
# Functions for this notebook
def read_libsvm_data(filepath: str) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    Reads libsvm file using cache
    :param filepath: the file path
    :return: the feature dataframe and target
    """
    data = load_svmlight_file('/dbfs' + filepath)
    X, y = pd.DataFrame(data[0].toarray()), data[1]
    return X, y


def stop_watch(func):
    """
    Measures the function execution time and prints it
    :param func: the executed function
    :return: the execution result
    """

    @wraps(func)
    def wrapper(*args, **kargs):
        start = time.time()
        result = func(*args, **kargs)
        elapsed_time = time.time() - start
        print(f"It took {elapsed_time} seconds to execute {func.__name__}.")
        return result

    return wrapper


@stop_watch
def feature_selection_chi2_sklearn(X: pd.DataFrame, y: np.ndarray,
                                   k: int) -> None:
    """
    Selects features based on chi square test by scikit-learn
    :param X: the feature dataframe
    :param y: the target array
    :param k: the number of features selected
    :return: None
    """
    selector = SelectKBest(chi2, k=k)
    selector.fit_transform(X, y)
    new_col1 = X.columns[selector.get_support()]
    print('Selected attribute: ', new_col1.to_list())


def read_libsvm_data_by_spark(filepath: str) -> pyspark.sql.DataFrame:
    """
    Reads libsvm file as spark dataframe
    :param filepath: the file path
    :return: the spark dataframe
    """
    spark = SparkSession.builder.appName("feature_selection").getOrCreate()
    df = spark.read.format("libsvm").load(filepath)
    return df


@stop_watch
def feature_selection_chi2_sparkml(df: pyspark.sql.DataFrame, k: int) -> None:
    """
    Selects features based on chi square test by Spark ML
    :param df: the spark dataframe
    :param k: the number of features selected
    :return: None
    """
    selector = ChiSqSelector(numTopFeatures=k, outputCol="selectedFeatures")
    model = selector.fit(df)
    print('Selected attribute: ', model.selectedFeatures)


@udf("map<long, double>")
def vector_as_mapUDF(v: pyspark.sql.Column) -> Dict[list, list]:
    """
    Extracts indices and values from SparseVector or DenseVector
    :param v: the spark dataframe's column
    :return: the map object of indices and values
    """
    if isinstance(v, SparseVector):
        return dict(zip(v.indices.tolist(), v.values.tolist()))
    elif isinstance(v, DenseVector):
        return dict(zip(range(len(v)), v.values.tolist()))


@udf("double")
def chi2UDF(observed: pyspark.sql.Column, freq: pyspark.sql.Column,
            total: pyspark.sql.Column) -> Column:
    """
    Calculates the chi squared value from the features
    :param observed: the observed feature column
    :param freq: the frequent feature column
    :param total: the total feature column
    :return: the chi squared value
    """
    expected = freq * total
    return math.pow(observed - expected, 2) / expected


@stop_watch
def feature_selection_chi2_sparse(df: pyspark.sql.DataFrame, k: int) -> None:
    """
    Selects features based on chi square test over the paper's idea
    :param df: the spark dataframe
    :param k: the number of features selected
    :return: None
    """
    # sum of all features
    feature_sum = (df
                   .select(explode(vector_as_mapUDF("features"))
                           .alias("feature", "value"))
                   .groupBy(col("feature"))
                   .sum("value")
                   .withColumnRenamed("sum(value)", "total"))

    # compute the observed statistics
    observed = (df
                .select("label", explode(vector_as_mapUDF("features"))
                        .alias("feature", "value"))
                .groupBy(col("label"), col("feature"))
                .sum("value")
                .withColumnRenamed("sum(value)", "observed"))

    # class (label) probabilities
    class_count = df.groupBy("label").count()
    total_count = class_count.agg(sum("count")).first()[0]
    class_prob = (class_count
                  .select(col("label"),
                          (col("count") / total_count).alias("freq")))

    # join the three dataframe
    oe = (observed
          .join(class_prob, "label")
          .join(feature_sum, "feature"))

    # compute chi-squared value and sort by it
    chisquare = (oe
                 .withColumn("chi2",
                             chi2UDF(col("observed"), col("freq"),
                                     col("total")))
                 .groupBy("feature")
                 .sum("chi2")
                 .sort(desc("sum(chi2)")))

    # print the selected features
    print('Selected attribute: ',
          chisquare.select(collect_list('feature')).first()[0][:k])


# Experiment 1 (single node)

In [ ]:
# Set the number of features selected
k = 5

## Dataset 1
Rows: 32,561 Features: 122

In [ ]:
# Scikit-Learn
X, y = read_libsvm_data(file1)
feature_selection_chi2_sklearn(X, y, k)

WARNING:root:[MemorizedFunc(func=<function read_libsvm_data at 0x7fd36953edc0>, location=./mycache/joblib)]: Clearing function cache identified by __main__--databricks-driver-<command-469973131362160>/read_libsvm_data
________________________________________________________________________________
[Memory] Calling __main__--databricks-driver-<command-469973131362160>.read_libsvm_data...
read_libsvm_data('/mnt/termpaper/a9a.txt')
_________________________________________________read_libsvm_data - 0.4s, 0.0min
Selected attribute: [38, 39, 41, 62, 74]
It took 0.034389495849609375 seconds to execute feature_selection_chi2_sklearn.

In [ ]:
# Read data as Spark Dataframe
df = read_libsvm_data_by_spark(file1)

In [ ]:
# Spark ML
feature_selection_chi2_sparkml(df, k)

Selected attribute: [0, 1, 2, 3, 4]
It took 1.2818901538848877 seconds to execute feature_selection_chi2_sparkml.

In [ ]:
# Sparse Version 
feature_selection_chi2_sparse(df, k)

Selected attribute: [39, 62, 38, 41, 74]
It took 4.378305196762085 seconds to execute feature_selection_chi2_sparse.

## Dataset 2
Rows: 748,401 Features: 1,163,024

In [ ]:
df = read_libsvm_data_by_spark(file2)

In [ ]:
feature_selection_chi2_sparkml(df, k)

Selected attribute: [3, 4, 7, 9, 10]
It took 117.66546678543091 seconds to execute feature_selection_chi2_sparkml.

In [ ]:
feature_selection_chi2_sparse(df, k)

Selected attribute: [10, 12, 9, 11, 3]
It took 35.72699975967407 seconds to execute feature_selection_chi2_sparse.

## Dataset 3
Rows: 8,407,752 Features: 20,216,830

In [ ]:
df = read_libsvm_data_by_spark(file3)

In [ ]:
feature_selection_chi2_sparkml(df, k)

Selected attribute: [1, 2, 3, 4, 5]
It took 3864.9682977199554 seconds to execute feature_selection_chi2_sparkml.

In [ ]:
feature_selection_chi2_sparse(df, k)

Selected attribute: [5430, 5428, 114, 440, 111]
It took 669.1588618755341 seconds to execute feature_selection_chi2_sparse.

# Experiment 2 (multi node)

## Dataset 2

In [ ]:
df = read_libsvm_data_by_spark(file2)

In [ ]:
feature_selection_chi2_sparkml(df, k)

Selected attribute: [3, 4, 7, 9, 10]
It took 115.18947196006775 seconds to execute feature_selection_chi2_sparkml.

In [ ]:
feature_selection_chi2_sparse(df, k)

Selected attribute: [10, 12, 9, 11, 3]
It took 55.53756499290466 seconds to execute feature_selection_chi2_sparse.

## Dataset 3

In [ ]:
df = read_libsvm_data_by_spark(file3)

In [ ]:
feature_selection_chi2_sparkml(df, k)

Selected attribute: [1, 2, 3, 4, 5]
It took 3085.272744178772 seconds to execute feature_selection_chi2_sparkml.

In [ ]:
feature_selection_chi2_sparse(df, k)

Selected attribute: [5430, 5428, 114, 440, 111]
It took 825.7812540531158 seconds to execute feature_selection_chi2_sparse.